# 1.0 Fix data

AIMS

En este archivo se realiza la limpieza de los archivos

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from mypackage import dir, dataquality
from mypackage.transforms import non_sex, diff_date, convert_variables

modality =  modality = 'p'
project = 'australian'
data = dir.make_dir_line(modality, project) 
raw = data('raw')
processed = data('processed')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

## 1.1 Fix data customers

In [3]:
df_clientes = pd.read_csv(raw / 'df_clientes.csv', sep = ',', decimal = '.', header = 0)
df_clientes['fecha_nacimiento'] = pd.to_datetime(df_clientes['fecha_nacimiento'], dayfirst=False, errors = 'coerce')
df_clientes = convert_variables(df_clientes, ['id_cliente', 'sexo'], 'str')
df_clientes = non_sex(df_clientes, 'sexo')
df_clientes = diff_date(df_clientes, "fecha_nacimiento", 'now')
dataquality.data_quality(df_clientes, threshold_discrete=10)

Execution data quality starts.

*****
The dataframe does not contain missing values.
*****
The dataframe does not contain duplicate values.
*****
The dataframe contains values that are not numeric.
Variable not numerit: id_cliente
Variable not numerit: nombre
Variable not numerit: fecha_nacimiento
*****
Discrete variable: sexo
*****
The dimension of dataframe is (500, 5)
*****

Execution data quality ends.



In [4]:
df_clientes.head()

,id_cliente,nombre,sexo,fecha_nacimiento,age_days
0,1001,Fredo Bonanno-Beccheria,1,1930-06-11,34247
1,1002,Ermenegildo Sansoni,1,1940-10-17,30466
2,1003,Antonia Bajardi,0,1916-06-15,39356
3,1004,Evangelista Biagi,0,1985-02-17,14272
4,1005,Iolanda Ceschi,0,2016-07-22,2794


## 1.2 Fix data products

In [5]:
df_productos = pd.read_csv(raw / 'df_productos.csv', sep = ',', decimal = '.', header = 0)
df_productos.rename(columns={'precio':'valor', 'production':'procentaje_produccion'}, inplace=True)
df_productos['valor_produccion'] = (df_productos['valor'] * df_productos['procentaje_produccion'])/100
dataquality.data_quality(df_productos, threshold_discrete=10)

Execution data quality starts.

*****
The dataframe does not contain missing values.
*****
The dataframe does not contain duplicate values.
*****
The dataframe contains values that are not numeric.
Variable not numerit: id_producto
Variable not numerit: articulo
Variable not numerit: category
Variable not numerit: color
Variable not numerit: L
Variable not numerit: tallas
*****
Discrete variable: tallas
*****
The dimension of dataframe is (398, 10)
*****

Execution data quality ends.



In [6]:
df_productos.head()

,id_producto,descripcion,articulo,category,valor,procentaje_produccion,color,L,tallas,valor_produccion
0,759ne,759,Short con tiritas,Short,40.0,46,Negro,ne,1 al 4,18.400
1,759bl,759,Short con tiritas,Short,40.0,46,Blanco,bl,1 al 4,18.400
2,759az,759,Short con tiritas,Short,40.0,46,Azul,az,1 al 4,18.400
3,759ro,759,Short con tiritas,Short,40.0,46,Rojo,ro,1 al 4,18.400
4,761ro,761,Short lienzo con bolsillo fruncido,Short,43.2,37,Rojo,ro,1 al 4,15.984


## 1.3 Fix data transactions

In [7]:
df_ventas = pd.read_csv(raw / 'df_ventas.csv', sep = ',', decimal = '.', header = 0)
df_ventas = df_ventas.loc[:,['id_factura', 'fecha', 'id_cliente', 'id_producto', 'quantity', 'id_store', 
                             'discount', 'per_discount2']]
df_ventas['fecha'] = pd.to_datetime(df_ventas['fecha'], dayfirst='false', errors = 'coerce')
df_ventas = convert_variables(df_ventas, ['id_factura', 'id_cliente', 'id_store', 'discount'], 
                                'str')
dataquality.data_quality(df_ventas, threshold_discrete=10)

Execution data quality starts.

*****
The dataframe does not contain missing values.
*****
The dataframe does not contain duplicate values.
*****
The dataframe contains values that are not numeric.
Variable not numerit: id_factura
Variable not numerit: fecha
Variable not numerit: id_cliente
Variable not numerit: id_producto
Variable not numerit: id_store
Variable not numerit: discount
*****
Discrete variable: quantity
*****
The dimension of dataframe is (9001, 8)
*****

Execution data quality ends.



In [8]:
df_ventas.head()

,id_factura,fecha,id_cliente,id_producto,quantity,id_store,discount,per_discount2
0,1001,2019-03-16,1331,1634ra,1,2,1,20.0
1,1002,2019-03-16,1131,5821gr,1,1,0,0.0
2,1003,2019-03-16,1344,9151ro,1,2,1,5.0
3,1004,2019-03-16,1444,9145gr,2,2,1,5.0
4,1005,2019-03-16,1345,1367ve,2,2,0,0.0


In [9]:
# Save data
df_clientes.to_parquet(processed/'df_clientes.parquet.gzip', compression='gzip')
df_productos.to_parquet(processed/'df_productos.parquet.gzip', compression='gzip')
df_ventas.to_parquet(processed/'df_ventas.parquet.gzip', compression='gzip')

In [10]:
print('ok_')

ok_
